# Exemple de finetuning d'un modèle BERT sur la classification des avis du site Allociné

[Ouvrir dans Google colab](https://colab.research.google.com/github/Herve07h22/nlp-fine-tuning/blob/main/allocine.ipynb#scrollTo=2zg2loyfZj3y)

In [ ]:
%pip install datasets transformers

In [1]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch

In [ ]:
raw_datasets = load_dataset("allocine")
raw_datasets.shape

## aperçu du dataset "Allociné"

5 commentaires positifs

In [ ]:
positive_samples = raw_datasets["train"].shuffle(seed=42).filter(lambda example: example['label']==1)
positive_samples['review'][:5]

5 commentaires négatifs

In [ ]:
positive_samples = raw_datasets["train"].shuffle(seed=42).filter(lambda example: example['label']==0)
positive_samples['review'][:5]

## Pre-processing du dataset

Pour l'entraînement du modèle de classification, on utiliser un `Trainer` de HuggingFace. Il utilise un dataset comportant 2 features :
- le label (0 ou 1)
- les input_ids tokenisés
- les attention_mask (1 si le mot est présent, 0 sinon)

On va donc pré-processer notre dataset avec le Tokeniser du modèle

In [20]:
tokenizer = AutoTokenizer.from_pretrained("camembert-base")

def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

tokenized_datasets.shape

100%|██████████| 20/20 [00:09<00:00,  2.13ba/s]


{'train': (160000, 4), 'validation': (20000, 4), 'test': (20000, 4)}